タイタニック号

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

titanic_df=pd.read_csv('titanic_train.csv')

In [ ]:
titanic_df.columns.values

In [ ]:
titanic_df

In [ ]:
カテゴリー変数の数を棒グラフで可視化

In [ ]:
print(titanic_df['Name'].value_counts(dropna=False))
#titanic_df['Name'].value_counts(dropna=False).plot.bar()

In [ ]:
print(titanic_df['Sex'].value_counts(dropna=False))
titanic_df['Sex'].value_counts(dropna=False).plot.bar()

In [ ]:
print(titanic_df['Ticket'].value_counts(dropna=False))
titanic_df['Ticket'].value_counts(dropna=False).plot.bar()

In [ ]:
print(titanic_df['Cabin'].value_counts(dropna=False))
titanic_df['Cabin'].value_counts(dropna=False).plot.bar()

In [ ]:
print(titanic_df['Embarked'].value_counts(dropna=False))
titanic_df['Embarked'].value_counts(dropna=False).plot.bar()

In [ ]:
欠損値の確認

In [ ]:
# 各列ごとの欠損値の数を表示
#missing_values = df_all.isnull().sum()
#print(missing_values)

# 各列の欠損値の数を計算
missing_values = titanic_df.isnull().sum()

# 欠損値がある列のみ表示
missing_values = missing_values[missing_values > 0]

# 結果を表示
print(missing_values)

In [ ]:
# 各列ごとの欠損値の割合を表示
missing_ratio = titanic_df.isnull().mean() * 100
# 欠損値がある列のみ表示
missing_ratio = missing_ratio[missing_ratio > 0]
print(missing_ratio)

欠損値の有無をヒートマップを用いて可視化

In [ ]:
import missingno as msno
msno.matrix(titanic_df.iloc[:,0:13],labels=True)

In [ ]:
カテゴリー変数をダミー変数化（ワンホットエンコーディング）

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# エンコードしたい列を指定
columns_to_encode = ['Sex','Embarked']  # エンコードしたい列名をリストで指定

# エンコードしない列を取得
non_encoded = titanic_df.drop(columns=columns_to_encode)

# OneHotEncoderのインスタンスを作成
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 指定した列のみをエンコード
encoded_array = encoder.fit_transform(titanic_df[columns_to_encode])

# エンコードされたデータをデータフレームに変換
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(columns_to_encode))

# int型に変換
encoded_df= encoded_df.astype('int64')

# 元のデータフレームに結合
new_titanic_df = pd.concat([non_encoded, encoded_df], axis=1)

# 結果を確認
print(new_titanic_df)
from IPython.display import display

#display(df_final)
# 元の列（category）を削除する
#欠損値が50%以上の列、使用しない名前を削除
new_titanic_df.drop(columns=['Name','Cabin','Ticket'], inplace=True)


# 結合したデータをCSVファイルに保存
new_titanic_df.to_csv('combined_file_2', index=False)


In [ ]:
new_titanic_df

new_titanic_dfに、欠損値が50%である列と、名前列が削除したデータが、combined_file_2.csv」に保存されている

欠損値補完

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier  # 例としてランダムフォレストを使用
from sklearn.impute import KNNImputer  # k-NN補完を使用する場合

# データの読み込み
new_titanic_df = pd.read_csv('combined_file_2.csv')
# 元のデータフレームのコピーを作成
original_data = new_titanic_df.copy()


# 特徴量とターゲットに分割
#X = df_all.drop.(columns=['id', 'ent aw']  # 例: 'ターゲット列名'を実際のターゲット列名に置き換え
X = new_titanic_df.drop(columns=['Survived'])
y = new_titanic_df['Survived']

# k-NN補完を適用する
imputer = KNNImputer(n_neighbors=5)
X_imputed = imputer.fit_transform(X)

# モデルの設定（例としてランダムフォレストを使用）
model = RandomForestClassifier(random_state=0)

# 交差検証の設定
k = 5  # 5分割交差検証
kf = KFold(n_splits=k, shuffle=True, random_state=0)

# 交差検証の実行
scores = cross_val_score(model, X_imputed, y, cv=kf, scoring='accuracy')

# 結果の表示
print(f"{k}-分割交差検証の各スコア: {scores}")
print(f"平均スコア: {np.mean(scores):.3f}")

df_imputed = pd.DataFrame(imputer.fit_transform(new_titanic_df), columns=new_titanic_df.columns)

# 元のデータを復元
new_titanic_df= original_data.copy()  # 元のデータに戻す

print("元のデータ:")
print(new_titanic_df)
# 補完後のデータをCSVファイルに保存
df_imputed.to_csv('in_k-NN_補完後データ.csv', index=False)

print("補完後のデータを'k-NN_補完後データ.csv'に保存しました。")

In [ ]:
補完前後のヒストグラム

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSVファイルからデータを読み込む
df = pd.read_csv('titanic_train.csv')  # ファイル名を適切に変更してください

# 欠損値のあるデータ
original_data = df[['Age']]  # 欠損値がある列を指定

# 補完後のデータ（例として平均値で補完）
imputed_df = pd.read_csv('in_k-NN_補完後データ.csv')

# 描画したい列をリストで指定する
selected_columns = ['Age']

# 元のデータのヒストグラムを描画
rows = (len(selected_columns) // 4) + 1  # 行数を計算
plt.figure(figsize=(18, 3 * rows))  # プロットのサイズを調整
for i, column in enumerate(selected_columns):
    plt.subplot(rows, 4, i + 1)  # 行列を調整
    plt.hist(original_data[column].dropna(), bins=30, alpha=0.7, color='blue', label='Original Data')
    plt.title(f'Histogram of {column}', fontsize=14)
    plt.xlabel('Value', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.legend()

plt.tight_layout()
plt.subplots_adjust(hspace=0.5, wspace=0.5)  # スペースを調整
plt.show()

# 補完されたデータのヒストグラムを描画
plt.figure(figsize=(18, 3 * rows))  # プロットのサイズを調整
for i, column in enumerate(selected_columns):
    plt.subplot(rows, 4, i + 1)  # 行列を調整
    plt.hist(imputed_df[column], bins=30, alpha=0.7, color='green', label='Imputed Data')  # dropna()を削除
    plt.title(f'Histogram of Imputed Data for {column}', fontsize=14)
    plt.xlabel('Value', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.legend()

plt.tight_layout()
plt.subplots_adjust(hspace=0.5, wspace=0.5)  # スペースを調整
plt.show()

In [ ]:
補完前後の箱ひげ図

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSVファイルからデータを読み込む
#df = pd.read_csv('in_機械学習用データ.csv')  # ファイル名を適切に変更してください

# 欠損値のあるデータ
#original_data = df[['apache 2', 'in alp', 'in plt sec', 'in plt inr', 'in crp', 'nibp', 'hr', 'in grip']]  # 欠損値がある列を指定

# 補完後のデータ
#imputed_df = pd.read_csv('in_k-NN_補完後データ.csv')

# 描画したい列をリストで指定する
#selected_columns = ['apache 2', 'in alp', 'in plt sec', 'in plt inr', 'in crp', 'nibp', 'hr',  'in grip']


# 元のデータの箱ひげ図を描画
plt.figure(figsize=(18, 3 * rows))  # プロットのサイズを調整
for i, column in enumerate(selected_columns):
    plt.subplot(rows, 4, i + 1)  # 行列を調整
    plt.boxplot(original_data[column].dropna(), vert=False, patch_artist=True, boxprops=dict(facecolor='blue', color='blue'), medianprops=dict(color='red'))
    plt.title(f'Boxplot of {column}', fontsize=14)
    plt.xlabel('Value', fontsize=12)

plt.tight_layout()
plt.subplots_adjust(hspace=0.5, wspace=0.5)  # スペースを調整
plt.show()

# 補完されたデータの箱ひげ図を描画
plt.figure(figsize=(18, 3 * rows))  # プロットのサイズを調整
for i, column in enumerate(selected_columns):
    plt.subplot(rows, 4, i + 1)  # 行列を調整
    plt.boxplot(imputed_df[column], vert=False, patch_artist=True, boxprops=dict(facecolor='green', color='green'), medianprops=dict(color='red'))
    plt.title(f'Boxplot of Imputed Data for {column}', fontsize=14)
    plt.xlabel('Value', fontsize=12)

plt.tight_layout()
plt.subplots_adjust(hspace=0.5, wspace=0.5)  # スペースを調整
plt.show()

データを訓練：テスト=8：2に分割

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # モデルは例としてRandom Forestを使用
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# データの読み込み
df = pd.read_csv('in_k-NN_補完後データ.csv')  # ファイル名を指定してください

# 繰り返し回数
n_splits = 10

# 評価結果を保存するリスト
scores_list = []

# 10回の分割と評価のループ
for i in range(n_splits):
    # データをシャッフルして分割
    train_set, test_set = train_test_split(df, test_size=0.2, random_state=i, stratify=df['Survived'])  # 'ent aw'は目的変数

    # 特徴量とラベルの設定
    X_train, y_train = train_set.drop(columns=['Survived']), train_set['Survived']
    X_test, y_test = test_set.drop(columns=['Survived']), test_set['Survived']
    
    # モデルの定義と学習
    model = RandomForestClassifier(random_state=42)  # 使用するモデルを設定
    model.fit(X_train, y_train)
    
    # 予測と評価
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # 評価結果を保存
    scores_list.append({
        'iteration': i,
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    })
# 結果をデータフレームにまとめる
scores_df = pd.DataFrame(scores_list)

# 平均的なスコアに近い分割を選ぶ
average_scores = scores_df[['accuracy', 'f1_score', 'precision', 'recall']].mean()
scores_df['score_difference'] = ((scores_df[['accuracy', 'f1_score', 'precision', 'recall']] - average_scores) ** 2).sum(axis=1)
best_split = scores_df.loc[scores_df['score_difference'].idxmin()]

# 選ばれた分割の情報を表示
print("平均的な性能に最も近い分割:")
print(best_split)

# 最も平均的な分割を再分割
train_set, test_set = train_test_split(df, test_size=0.2, random_state=int(best_split['iteration']), stratify=df['Survived'])

# 結果を保存
train_set.to_csv('selected_train_set.csv', index=False)
test_set.to_csv('selected_test_set.csv', index=False)
print("最も平均的な分割のトレーニングセットとテストセットが保存されました。")


In [ ]:
統計量、p値の出力

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
from scipy.stats import mannwhitneyu, chi2_contingency

# データの読み込み
imputed_train_df = pd.read_csv('selected_train_set.csv', header=0)
imputed_test_df = pd.read_csv('selected_test_set.csv', header=0)

# 連続変数とカテゴリ変数の範囲を指定
imputed_train_part = imputed_train_df.iloc[:, 1:4].dropna()
imputed_test_part = imputed_test_df.iloc[:, 1:4].dropna()
imputed_train_cate = imputed_train_df.iloc[:, 4:14].dropna()
imputed_test_cate = imputed_test_df.iloc[:, 4:14].dropna()

# 統計量を計算する関数
def calculate_statistics(df):
    quantiles = df.quantile([0.25, 0.50, 0.75])
    result_df = pd.DataFrame({
        'Median': quantiles.loc[0.50],
        'IQR': quantiles.loc[0.75] - quantiles.loc[0.25],
        '25th Percentile': quantiles.loc[0.25],
        '75th Percentile': quantiles.loc[0.75]
        #'Range': quantiles.loc[0.25].astype(str) + ' - ' + quantiles.loc[0.75].astype(str)  # 四分位範囲の「ここからここまで」表記
        # Rangeの生成部分
        #'Range': quantiles.loc[0.25].map('{:.2f}'.format) + ' - ' + quantiles.loc[0.75].map('{:.2f}'.format)

    })
    return result_df

# 連続変数の統計量計算
train_statistics = calculate_statistics(imputed_train_part).add_prefix('Train_')
test_statistics = calculate_statistics(imputed_test_part).add_prefix('Test_')
imputed_test_train_result_df = pd.concat([train_statistics, test_statistics], axis=1)

# U検定でのp値計算
p_values = []
for column in imputed_train_part.columns:
    train_data = imputed_train_part[column]
    test_data = imputed_test_part[column]
    _, p_value = mannwhitneyu(train_data, test_data)
    p_values.append(p_value)
imputed_test_train_result_df['P-Value'] = p_values

# カテゴリ変数の数と割合を計算する関数
def get_categorical_summary(df, prefix):
    counts = df.sum()
    percentages = (counts / len(df)) * 100
    return pd.DataFrame({f'{prefix}_Count': counts, f'{prefix}_Percentage': percentages})
# 訓練データとテストデータのカテゴリ変数の集計
train_categorical_summary = get_categorical_summary(imputed_train_cate, 'Train')
test_categorical_summary = get_categorical_summary(imputed_test_cate, 'Test')
imputed_df_categorical = pd.concat([train_categorical_summary, test_categorical_summary], axis=1)

# カテゴリ変数用のp値（カイ二乗検定）
chi2_p_values = []
for column in imputed_train_cate.columns:
    contingency_table = pd.crosstab(imputed_train_cate[column], imputed_test_cate[column])
    _, p, _, _ = chi2_contingency(contingency_table)
    chi2_p_values.append(p)
imputed_df_categorical['P-Value'] = chi2_p_values

# 結果を統合
final_summary = {
    'Variable': [],
    'Type': [],
    'Train': [],
    'Test': [],
    'P-Value': []
}

# 連続変数の結果を追加
for column in imputed_test_train_result_df.index:
    final_summary['Variable'].append(column)
    final_summary['Type'].append('Continuous')
    final_summary['Train'].append(f"{imputed_test_train_result_df['Train_Median'].loc[column]} ")
    final_summary['Test'].append(f"{imputed_test_train_result_df['Test_Median'].loc[column]} ")
    final_summary['P-Value'].append(imputed_test_train_result_df['P-Value'].loc[column])

# カテゴリ変数の結果を追加
for column in imputed_df_categorical.index:
    final_summary['Variable'].append(column)
    final_summary['Type'].append('Categorical')
    final_summary['Train'].append(f"{imputed_df_categorical['Train_Count'].loc[column]} ({imputed_df_categorical['Train_Percentage'].loc[column]:.2f}%)")
    final_summary['Test'].append(f"{imputed_df_categorical['Test_Count'].loc[column]} ({imputed_df_categorical['Test_Percentage'].loc[column]:.2f}%)")
    final_summary['P-Value'].append(imputed_df_categorical['P-Value'].loc[column])
# データフレームに変換
final_summary_df = pd.DataFrame(final_summary)

# 表の表示
print(final_summary_df)

# 小数点以下を2桁にフォーマット
final_summary_df['Train'] = final_summary_df['Train'].apply(lambda x: f"{x:.2f}" if isinstance(x, float) else x)
final_summary_df['Test'] = final_summary_df['Test'].apply(lambda x: f"{x:.2f}" if isinstance(x, float) else x)
final_summary_df['P-Value'] = final_summary_df['P-Value'].apply(lambda x: f"{x:.2f}" if isinstance(x, float) else x)
# CSVファイルに保存
final_summary_df.to_csv('in_selected_final_summary_table_mix.csv', index=False, encoding='utf-8-sig')
